# MiniCPM-o-2.6 多模态语音能力演示

本 Notebook 演示了 MiniCPM-o-2.6 模型的各种语音功能，包括：
- 语音模仿
- 音频助手对话
- 音频理解
- 语音生成
- 语音克隆

## 1. 环境准备和模型加载

In [ ]:
import torch
from PIL import Image
from modelscope import AutoModel, AutoTokenizer
import librosa
import IPython.display as ipd
import os

In [ ]:
# 模型路径
model_path = './models/MiniCPM-o-2_6'

# 加载模型
model = AutoModel.from_pretrained(
    model_path,
    trust_remote_code=True,
    attn_implementation='sdpa',
    torch_dtype=torch.bfloat16,
    init_vision=True,
    init_audio=True,
    init_tts=True
)

model = model.eval().cuda()
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# 初始化 TTS
model.init_tts()

print("模型加载完成!")

## 2. 语音模仿演示

In [ ]:
# 语音模仿功能
def audio_mimick_demo(audio_path):
    """语音模仿演示"""
    if not os.path.exists(audio_path):
        print(f"音频文件不存在: {audio_path}")
        return
    
    # 加载音频
    audio_input, _ = librosa.load(audio_path, sr=16000, mono=True)
    
    # 设置提示词
    mimick_prompt = "请重复每个用户的讲话内容，包括语音风格和内容。"
    
    # 构建消息
    msgs = [{'role': 'user', 'content': [mimick_prompt, audio_input]}]
    
    # 生成响应
    res = model.chat(
        msgs=msgs,
        tokenizer=tokenizer,
        sampling=True,
        max_new_tokens=128,
        use_tts_template=True,
        temperature=0.3,
        generate_audio=True,
        output_audio_path='mimick_output.wav',
    )
    
    print(f"识别文本: {res.text}")
    
    # 播放原始音频
    print("原始音频:")
    display(ipd.Audio(audio_path))
    
    # 播放模仿音频
    print("模仿音频:")
    display(ipd.Audio('mimick_output.wav'))
    
    return res

# 使用示例（需要提供音频文件）
# audio_mimick_demo('path/to/your/audio.wav')

## 3. 音频助手对话演示

In [ ]:
# 音频助手对话功能
def audio_assistant_demo(ref_audio_path, question_audio_path, language='zh'):
    """音频助手对话演示"""
    if not os.path.exists(ref_audio_path) or not os.path.exists(question_audio_path):
        print("音频文件不存在")
        return
    
    # 加载参考音频
    ref_audio, _ = librosa.load(ref_audio_path, sr=16000, mono=True)
    
    # 获取系统提示
    sys_prompt = model.get_sys_prompt(
        ref_audio=ref_audio, 
        mode='audio_assistant', 
        language=language
    )
    
    # 加载问题音频
    question_audio, _ = librosa.load(question_audio_path, sr=16000, mono=True)
    user_question = {'role': 'user', 'content': [question_audio]}
    
    # 构建消息
    msgs = [sys_prompt, user_question]
    
    # 生成响应
    res = model.chat(
        msgs=msgs,
        tokenizer=tokenizer,
        sampling=True,
        max_new_tokens=128,
        use_tts_template=True,
        generate_audio=True,
        temperature=0.3,
        output_audio_path='assistant_response.wav',
    )
    
    print(f"助手响应: {res.text}")
    
    # 播放问题音频
    print("问题音频:")
    display(ipd.Audio(question_audio_path))
    
    # 播放助手响应
    print("助手响应音频:")
    display(ipd.Audio('assistant_response.wav'))
    
    return res

# 使用示例（需要提供音频文件）
# audio_assistant_demo('ref_audio.wav', 'question.wav')

## 4. 音频理解演示

In [ ]:
# 音频理解功能
def audio_understanding_demo(audio_path, task_prompt="Summarize the main content of the audio."):
    """音频理解演示"""
    if not os.path.exists(audio_path):
        print(f"音频文件不存在: {audio_path}")
        return
    
    # 加载音频
    audio_input, _ = librosa.load(audio_path, sr=16000, mono=True)
    
    # 构建消息
    msgs = [{'role': 'user', 'content': [task_prompt, audio_input]}]
    
    # 生成响应
    res = model.chat(
        msgs=msgs,
        tokenizer=tokenizer,
        sampling=True,
        max_new_tokens=128,
        use_tts_template=True,
        generate_audio=True,
        temperature=0.3,
        output_audio_path='understanding_result.wav',
    )
    
    print(f"理解结果: {res.text}")
    
    # 播放原始音频
    print("原始音频:")
    display(ipd.Audio(audio_path))
    
    # 播放理解结果音频
    print("理解结果音频:")
    display(ipd.Audio('understanding_result.wav'))
    
    return res

# 使用示例（需要提供音频文件）
# audio_understanding_demo('path/to/audio.mp3')

## 5. 语音生成演示

In [ ]:
# 语音生成功能
def voice_generation_demo(instruction_prompt):
    """语音生成演示"""
    # 构建消息
    msgs = [{'role': 'user', 'content': [instruction_prompt]}]
    
    # 生成响应
    res = model.chat(
        msgs=msgs,
        tokenizer=tokenizer,
        sampling=True,
        max_new_tokens=128,
        use_tts_template=True,
        generate_audio=True,
        temperature=0.3,
        output_audio_path='generated_voice.wav',
    )
    
    print(f"生成指令: {instruction_prompt}")
    print(f"生成文本: {res.text}")
    
    # 播放生成的音频
    print("生成的音频:")
    display(ipd.Audio('generated_voice.wav'))
    
    return res

# 示例：中文语音生成
chinese_instruction = '在新闻中，一个年轻男性兴致勃勃地说："祝福亲爱的祖国母亲美丽富强！"他用低音调和低音量，慢慢地说出了这句话。'
voice_generation_demo(chinese_instruction)

## 6. 语音克隆演示

In [ ]:
# 语音克隆功能
def voice_cloning_demo(ref_audio_path, text_to_speak, language='zh'):
    """语音克隆演示"""
    if not os.path.exists(ref_audio_path):
        print(f"参考音频文件不存在: {ref_audio_path}")
        return
    
    # 加载参考音频
    ref_audio, _ = librosa.load(ref_audio_path, sr=16000, mono=True)
    
    # 获取系统提示
    sys_prompt = model.get_sys_prompt(
        ref_audio=ref_audio, 
        mode='voice_cloning', 
        language=language
    )
    
    # 构建用户消息
    text_prompt = "Please read the text below."
    user_question = {'role': 'user', 'content': [text_prompt, text_to_speak]}
    
    # 构建消息
    msgs = [sys_prompt, user_question]
    
    # 生成响应
    res = model.chat(
        msgs=msgs,
        tokenizer=tokenizer,
        sampling=True,
        max_new_tokens=128,
        use_tts_template=True,
        generate_audio=True,
        temperature=0.3,
        output_audio_path='cloned_voice.wav',
    )
    
    print(f"要克隆的文本: {text_to_speak}")
    print(f"克隆结果: {res.text}")
    
    # 播放参考音频
    print("参考音频:")
    display(ipd.Audio(ref_audio_path))
    
    # 播放克隆音频
    print("克隆音频:")
    display(ipd.Audio('cloned_voice.wav'))
    
    return res

# 使用示例（需要提供参考音频文件）
# voice_cloning_demo('ref_audio.wav', '全名制作人们大家好，我是小黑子，鸡你太美')

## 7. 综合演示

运行以下代码来测试所有功能（需要提供相应的音频文件）：

In [ ]:
# 综合演示函数
def run_all_demos():
    """运行所有演示（需要提供音频文件）"""
    print("=== MiniCPM-o-2.6 多模态语音能力综合演示 ===")
    
    # 1. 语音生成演示
    print("\n1. 语音生成演示")
    instruction = '一位温柔的女性用甜美的声音轻声说道："今天天气真好，适合出去散步。"她的语调平缓，充满了温暖。'
    voice_generation_demo(instruction)
    
    # 其他演示需要音频文件，请根据实际情况取消注释并提供文件路径
    
    # 2. 语音模仿演示
    # print("\n2. 语音模仿演示")
    # audio_mimick_demo('path/to/mimick_audio.wav')
    
    # 3. 音频理解演示
    # print("\n3. 音频理解演示")
    # audio_understanding_demo('path/to/understanding_audio.mp3')
    
    # 4. 音频助手演示
    # print("\n4. 音频助手演示")
    # audio_assistant_demo('path/to/ref_audio.wav', 'path/to/question.wav')
    
    # 5. 语音克隆演示
    # print("\n5. 语音克隆演示")
    # voice_cloning_demo('path/to/ref_audio.wav', '要克隆的文本内容')
    
    print("\n=== 演示完成 ===")

# 运行演示
run_all_demos()